In [6]:
import nltk, os, math, csv
from nltk.sem.logic import printtype
from nltk import tokenize
from nltk.util import ngrams

## Contagem de palavras

In [13]:
# Abre o corpus específico
# O corpus específico pode estar dividido em vários arquivos TXTs. Em uma mesma pasta
# MAC
caminho_corpus_especifico = "/Users/bertolo/Dropbox/retorica_algoritimica/comentarios_elis_texto_unico.txt"
# Windows
# Corpus_especifico =  "C:\\Users\\ferna\\Dropbox\\retorica_algoritimica\\corpus_comentarios_elis"
# Caminho_corpus_especifico = "C:\\Users\\ferna\\OneDrive\\Escritorio\\corpus_especifico.txt"

# Abrindo o corpus geral
caminho_corpus_geral = "pt.txt"

In [8]:
# Conjunto de funções básicas / abre arquivo, faz o loglikelihood, calibra o corpus loglikelihood

def itera_ou_abre_arquivos(caminho_corpus):
    """
    Verifica se o caminho fornecidos leva a um txt, ou a diretório
    contendo arquivos txt. Retorna uma lista com os caminhos.

    Args:
        caminho_corpus(str): Caminho para um arquivo ou diretório específico.

    Returns:
        (str): O curpus é unido em uma única string
    """
    corpora = []
    # Se for um arquivo .txt, lê diretamente
    if caminho_corpus.endswith(".txt"):
        try:
            with open(caminho_corpus, "r", encoding="utf-8") as arquivo:
                corpora.append(arquivo.read())
        except OSError as e:
            print(f"Erro ao abrir o arquivo {caminho_corpus}: {e}")

    # Se for um diretório, lê todos os arquivos .txt dentro dele
    else:
        try:
            for nome_arquivo in os.listdir(caminho_corpus):
                if nome_arquivo.endswith(".txt"):
                    caminho_completo = os.path.join(caminho_corpus, nome_arquivo)
                    try:
                        with open(caminho_completo, "r", encoding="utf-8") as arquivo:
                            corpora.append(arquivo.read())
                    except OSError as e:
                        print(f"Erro ao abrir o arquivo {caminho_completo}: {e}")
        except OSError as e:
            print(f"Erro ao acessar o diretório {caminho_corpus}: {e}")
    corpora = " ".join(corpora)
    print(f"Nº Caracteres corpora: {caminho_corpus} - \n{len(corpora)}")
    return corpora

def string_to_freqdist(corpus):
    """ Transforma uma string em um objeto frqFist

    Args:
        corpus (str) - um objeto string

    Return:
        freqdist - FreqDist é uma subclasse de collections.Counter, ou seja, age como um contador de elementos.

        Métodos úteis:
        fdist.most_common(n): Retorna as n palavras mais frequentes.
        fdist.freq(palavra): Retorna a frequência relativa de uma palavra (proporção no total).
        fdist.plot(n): Plota um gráfico das n palavras mais frequentes.
        fdist.N(): Retorna o número total de palavras.
        fdist.keys(): Retorna as palavras únicas encontradas.

        Se a coleção de textos for muito grande, ele vai apresentar um resumo, como este:
        <FreqDist with 13280 samples and 67581 outcomes>
        Isso significa 13280 palavras únicas e 68581 palavras no total
     """

    # Cria o objeto token_espaco, da classe WhitespaveTokenizer,
    # Esse modulo tokenize, separa a string única em strings menores
    token_espaco = tokenize.WhitespaceTokenizer()
    corpus_tokenizado = token_espaco.tokenize(corpus)
    corpus_freqdist = nltk.FreqDist(word.lower() for word in corpus_tokenizado)

    return corpus_freqdist

def log_likelihood(oc, og, n, ng):
    """ Retorna um inteiro com o valor do loglikelihood

    Args:
        OC (int) - Ocorrencias no Corpus específico
        OG (int) - Ocorrencias no corpus Geral
        N  (int) - Tamanho do corpus especifico
        NG (int) - Tamanho do corpus geral

    Return - Int: resultado da verosemelhança

        Quanto maior o resultado retornado, menor a semelhança na distribuição das ocorrências.
        0, por exemplo, significaria uma distribuição exatamante igual.
        Esta função é importante para a função corpus_calibrado_log_likelihood()
    """

    taxa_ocorrencias = og / ng
    # OE - Ocerrencias esperadas com base no corpus geral
    oe = 1 if taxa_ocorrencias * n <= 0 else taxa_ocorrencias * n
    likelihood = 2 * (oc * math.log(oc / oe) + (n - oc) * (math.log((n - oc) / (n - oe))))

    return likelihood

def corpus_calibrado_log_likelihood(c_especifico_freqDist, c_geral_freqDist, limiar, stopwords=["a", "e", "i", "o", "u", ",", "?", "!"]):
    """ Entram dois corpora com freq e dist e sai um corpus com freq dist já calibrado

    Args
    # c_especifico_freqDist [freqDist] - Objeto contendo o Corpus específico
    # c_geral_freqDist [freqDist] - Objeto contendo uma amostra geral da língua
    # limiar [int] - Tolerância sobre o grau de semelhança. 0 = Mesma proporção. Bons valores variam de 1250 a 1750
    # stropwords [list] - Lista de palavras que deve ser excluidas do corpus de saída, independente do grau de varosssemelhança.

    Return - Um objeto freqdist

    A função compara o valor do loglikelihood, usando a função log_likelihood(), das ocorrencias de cada palavra do corpus específico no corpus geral.
    Se o valor estiver a baixo do limiar, a palavra é excluida da amostra.
    """

    c_especifico_calibrado = {}
    for palavra, freq in c_especifico_freqDist.items():
        if log_likelihood(freq, c_geral_freqDist[palavra], c_especifico_freqDist.N(),
                          c_geral_freqDist.N()) >= limiar and palavra not in stopwords:
            c_especifico_calibrado[palavra] = freq  # Corrigindo a atribuição ao dicionário

    c_especifico_calibrado = nltk.FreqDist(c_especifico_calibrado)
    return c_especifico_calibrado

In [9]:
# Função que egloba todas as funções da anterior. Entra uma string e retorna uma objeto freqDist (calibrado).
def lista_de_mais_frequentes_calibrada(caminho_corpus_especifico, caminho_corpus_geral, limiar, stopwords=["a", "e", "i", "o", "u", ",", "?", "!"]):
    """
        Abre duas str com o local dos corpora, sejam arquivos ou diretórios
        processa e devolve um arquivo FreqDist já com as palavras que estão acima do limiar retiradas.

    Args:
        caminho_corpus_especifico (str):
        caminho_corpus_geral (str):
        limiar (int):
        stopwords (list):

    return:
        (nltk.probability.FreqDist): Um objeto freqdist do corpus específico já calibrado na probabilidade.

        Métodos úteis:
        fdist.most_common(n): Retorna as n palavras mais frequentes.
        fdist.freq(palavra): Retorna a frequência relativa de uma palavra (proporção no total).
        fdist.plot(n): Plota um gráfico das n palavras mais frequentes.
        fdist.N(): Retorna o número total de palavras.
        fdist.keys(): Retorna as palavras únicas encontradas.

    A função compara o valor do loglikelihood, usando a função log_likelihood(), das ocorrencias de cada palavra do corpus específico no corpus geral.
    Se o valor estiver a baixo do limiar, a palavra é excluida da amostra.

     A função Depende de várias funções anteriores, como:
     itera_ou_abre_arquivos()
     string_to_freqdist()
     log_likelihood()
     corpus_calibrado_log_likelihood()
    """

    #
    # Entra o caminho dos corpus, específico e geral, e sai o corpus freqDist específico já calibrado

    # Entram dois caminhos de arquivos
    # Sai um objeto nltk.probability.FreqDist

    # strings
    corpus_especifico = itera_ou_abre_arquivos(caminho_corpus_especifico)
    corpus_geral = itera_ou_abre_arquivos(caminho_corpus_geral)

    # String freqdist
    c_especifico_tokenizado = string_to_freqdist(corpus_especifico)
    c_geral_tokenizado = string_to_freqdist(corpus_geral)

    # Chama a função corpus freqdist calibrado
    corpus_calibrado = corpus_calibrado_log_likelihood(c_especifico_tokenizado, c_geral_tokenizado, limiar, stopwords)
    return corpus_calibrado

In [10]:
# Entra um corpus freqDist e salva um .cvs com os x primeiros resultados
def salva_csv_de_corpus_freq_dist(corpus_freqDist, entradas=10, nome_do_arquivo="corpus.csv", delimitador=";"):
    """ Transforma um objeto freqdist em um arquivo csv com uma lista dos N mais frequentes

        args:
            corpus_freqDist(freqdist): freqDist é uma subclasse de collections.Counter, ou seja, age como um contador de elementos.
            entradas(int): Número de linhas que o csv vai ter
            nome_do_arquivo(str): É bom que ele termine com a extenção csv
            delimitador(str): O Caractere que vai ser usado para separar os campos

        return:
            Salva um arquivo .csv na mesma pasta do script

        Dica: Caso se queira usar um "tab" como delimitador, o caractere é o "\t"

            """
    # Cabeçalho do csv
    dados = [["N","Palavra", "Frequência"]]
    counter = 1

    for key, freq in corpus_freqDist.most_common(entradas):
        linha = [counter, key, freq]
        counter += 1
        dados.append(linha)
    # Criando e salvando o arquivo CSV
    with open(nome_do_arquivo, "w", newline="", encoding="utf-8") as arquivo_csv:
        escritor = csv.writer(arquivo_csv, delimiter=delimitador, quotechar='"', quoting=csv.QUOTE_MINIMAL)
        escritor.writerows(dados)

    print("CSV salvo com sucesso!")

In [6]:
# Salva os csv dos corpora: especifico, geral e específico filtrado 

# ESPECÍFICO
# 1 - Abre o corpus e transforma em (str)
corpus_especifico = itera_ou_abre_arquivos(caminho_corpus_especifico)
# 2 - Transfor a string em um objeto freqDist
corpus_especifico_freqDist = string_to_freqdist(corpus_especifico)
# 3 - Salva os 16 termos mais frequentes
salva_csv_de_corpus_freq_dist(corpus_especifico_freqDist, 16, "corpus_especifico_v02.csv", "\t")

# GERAL
# 1 - Abre o corpus e transforma em (str)
corpus_geral = itera_ou_abre_arquivos(caminho_corpus_geral)
# 2 - Transfor a string em um objeto freqDist
corpus_geral_freqDist = string_to_freqdist(corpus_geral)
# 3 - Salva os 16 termos mais frequentes
salva_csv_de_corpus_freq_dist(corpus_geral_freqDist, 16, "copacabana_v02.csv", "\t")

# ESPECÍCO FILTRADO
# 1
corpus_geral_filtrado = lista_de_mais_frequentes_calibrada(caminho_corpus_especifico,caminho_corpus_geral,900)
salva_csv_de_corpus_freq_dist(corpus_geral_filtrado,16,"copacabana_filtrado.csv", "\t")

Nº Caracteres corpora: /Users/bertolo/Dropbox/retorica_algoritimica/comentarios_elis_texto_unico.txt - 
411650
CSV salvo com sucesso!
Nº Caracteres corpora: pt.txt - 
17034654
CSV salvo com sucesso!
Nº Caracteres corpora: /Users/bertolo/Dropbox/retorica_algoritimica/comentarios_elis_texto_unico.txt - 
411650
Nº Caracteres corpora: pt.txt - 
17034654
CSV salvo com sucesso!


In [11]:
# Imprime os temos mais comuns em um corpus calibrados


string = itera_ou_abre_arquivos(caminho_corpus_especifico)
fq = lista_de_mais_frequentes_calibrada(caminho_corpus_especifico,caminho_corpus_geral,1400)
fq.most_common(20)

Nº Caracteres corpora: /Users/bertolo/Dropbox/retorica_algoritimica/comentarios_elis_texto_unico.txt - 
411650
Nº Caracteres corpora: /Users/bertolo/Dropbox/retorica_algoritimica/comentarios_elis_texto_unico.txt - 
411650
Nº Caracteres corpora: pt.txt - 
17034654


[('parabéns', 779),
 ('propaganda', 610),
 ('comercial', 574),
 ('elis', 465),
 ('❤', 364),
 ('lindo', 329),
 ('volkswagen', 267),
 ('linda', 251),
 ('vw', 248),
 ('emocionante', 208),
 ('ditadura', 144),
 ('regina', 139),
 ('você', 26)]

## Concordanciadores

In [9]:
# Concordanciador de uma palavra chave com o corpus em um único texto

# Arquivo para string
corpus_especifico = itera_ou_abre_arquivos(caminho_corpus_especifico)

# entra uma string tokenizada por palavras
# Cria o token
token_espaco = tokenize.WhitespaceTokenizer()
# Tokeniza o texto
corpus_tokenizado = token_espaco.tokenize(corpus_especifico)

corpus_textizado = nltk.Text(corpus_tokenizado)

# Gerar a lista de concordância para a palavra "whale"
corpus_textizado.concordance("elis", width=200, lines=100)

Nº Caracteres corpora: /Users/bertolo/Dropbox/retorica_algoritimica/comentarios_elis_texto_unico.txt - 
411650
Displaying 100 of 465 matches:
 Não vi nada demais. Talvez pq seja feito com base em um certo culto que eu nunca entendi bem pq, Elis Regina (mediazinha) cantando a música mais chinfrim do Belchior 🥰☺️ Vim do twitter só pra conferi
ra Encantador, trabalho belissimo. Parabens a todos envolvidos. 🎉❤ Genial!! Tive Fuscas e Kombis. Elis em IA com Maria Rita ❤❤❤❤ 🌾🍂☕🌾🍃🦋 Muito boa..... gostei demais Uau!!!! 🤩🤩🤩🤩👏🏼👏🏼👏🏼👏🏼👏🏼👏🏼🥹🥹🥹🥹🥹🥹🥹🥲🥲🥲❤
ís e nossa história ❤ Eu não gostei. Eu Asmei!!!!!! 😍😍😍 Q bizarro... Essa cara computadorizada da elis regina pra vender carro..🤦🏿‍♀️🤦🏿‍♀️🤦🏿‍♀️ Uau, que comercial lindo! Foi de uma delicadeza trazer c
que comercial lindo! Foi de uma delicadeza trazer cenas cotidianas da vida, a música escolhida, a Elis e Maria Rita, sensacional! Parabéns, de arrepiar!! 👏🏻 Volks, mais bonita que a Elis é a nova Komb
 escolhida, a Elis e Maria Rita, sensacional! Parabéns

In [10]:
# Concordanciador de uma única palavra chave com o corpus dividido em uma lista de strings

pasta_corpus = "corpus_comentarios_elis/"

# Para cada arquivo dentro de uma pasta
for arquivo in os.listdir(pasta_corpus):
    # Verifica se o arquivo termina com txt
    if arquivo.endswith(".txt"):
        caminho_arquivo = os.path.join(pasta_corpus, arquivo)

        # Abre o arquivo e tranforma o texto em uma string
        with open(caminho_arquivo, "r", encoding="utf-8") as f:
            texto = f.read().strip()  # Remove espaços extras

        # Tokeniza a sentença
        token_espaco = tokenize.WhitespaceTokenizer()
        corpus_token = token_espaco.tokenize(texto)

        # Cria o objeto Text do NLTK
        corpus_text = nltk.Text(corpus_token)

        # Exibe concordância se a palavra alvo existir
        palavra_alvo = "Elis"
        if palavra_alvo in corpus_token:
            corpus_text.concordance(palavra_alvo, width=100)

Displaying 1 of 1 matches:
 em um certo culto que eu nunca entendi bem pq, Elis Regina (mediazinha) cantando a música mais chin
Displaying 1 of 1 matches:
                 Genial!! Tive Fuscas e Kombis. Elis em IA com Maria Rita ❤❤❤❤
Displaying 1 of 1 matches:
cenas cotidianas da vida, a música escolhida, a Elis e Maria Rita, sensacional!
Displaying 1 of 1 matches:
                       Volks, mais bonita que a Elis é a nova Kombi. Incrível. :D
Displaying 1 of 1 matches:
                                                Elis Regina jamais faria tal comercial!
Displaying 1 of 1 matches:
abéns pelo belo comercial e a ótima homenagem a Elis Regina e pela escolha da música de Belchior .
Displaying 1 of 1 matches:
que na época apoiou a ditadutra militar (a qual Elis Regina fazia oposição) e entregava opositores a
Displaying 1 of 1 matches:
                                              A Elis Regina se retorcendo no caixão vendo sua filha 
Displaying 1 of 1 matches:
o na produção de qualquer 

In [22]:
# Lista de palavras mais comuns apenas nas sentenças que tiverem a palavra alvo

# Pasta onde estão os arquivos
pasta_corpus = "corpus_comentarios_elis/"
# Palavra alvo
palavra_alvo = "elis"
# Variável que armazenará o textos das sentenças que contenham a palavra alvo em uma única string
corpus_para_analize_da_palavra_alvo = ""
contador = 0
# Para cada arquivo dentro de uma pasta
for arquivo in os.listdir(pasta_corpus):
    if arquivo.endswith(".txt"):
        caminho_arquivo = os.path.join(pasta_corpus, arquivo)

        # Abre e lê o arquivo
        with open(caminho_arquivo, "r", encoding="utf-8") as f:
            texto = f.read().strip()  # Remove espaços extras

        # Tokeniza a sentença
        token_espaco = tokenize.WhitespaceTokenizer()
        corpus_token = token_espaco.tokenize(texto)


        # Se a sentença tiver a palavra alvo
        if palavra_alvo in corpus_token:
            contador +=1
            # Adciona na strin geral
            corpus_para_analize_da_palavra_alvo += " " + texto.lower()

print(contador)

# Cria o freqdist das sentenças selecionadas
corpus_da_palavra_alvo = string_to_freqdist(corpus_para_analize_da_palavra_alvo)


# Cria o freqdist da amostra geral da língua portuguesa
amostra_geral_lingua = itera_ou_abre_arquivos(caminho_corpus_geral)
amostra_geral_lingua = string_to_freqdist(amostra_geral_lingua)

# seleciona apenas as palavras acima do limiar do loglikelihood
Lista_mais_comuns_da_palvra_alvo = corpus_calibrado_log_likelihood(corpus_da_palavra_alvo, amostra_geral_lingua, 40)

# Cria a lista de mais comuns
Lista_mais_comuns_da_palvra_alvo.most_common(15)

 #corpus_freqdist = nltk.FreqDist(word.lower() for word in corpus_tokenizado)

30
Nº Caracteres corpora: pt.txt - 
17034654


[('elis', 32),
 ('regina', 10),
 ('ditadura', 8),
 ('empresa', 6),
 ('propaganda', 5),
 ('imagem', 5),
 ('apoiou', 4),
 ('patrocinou', 3),
 ('volkswagen', 3)]

In [4]:
# Sorteia x Comentários que contenham uma determinada palavra

import random, os

pasta_corpus = "corpus_comentarios_elis/"
palavra_alvo = "Elis"

lista_de_comentarios_selecionados = []

# Para cada arquivo dentro de uma pasta
for arquivo in os.listdir(pasta_corpus):
    # Verifica se o arquivo termina com txt
    if arquivo.endswith(".txt"):
        caminho_arquivo = os.path.join(pasta_corpus, arquivo)

        # Abre o arquivo e tranforma o texto em uma string
        with open(caminho_arquivo, "r", encoding="utf-8") as f:
            comentario = f.read().strip()  # Remove espaços extras

        if palavra_alvo.lower() in comentario.lower():
            lista_de_comentarios_selecionados.append(comentario)

# Sorteando 10 itens sem repetição
itens_sorteados = random.sample(lista_de_comentarios_selecionados, 5)

print(itens_sorteados)


['Nem sou do tempo de Elis Regina e me emocionei com a propaganda, ficou perfeita', 'Horrível.\nacho uma falta de respeito com a imagem da Elis.\nPoderiam colocar um vídeo real dela cantando a Elis não é uma inteligência artificial nunca deve se comparar como tal\nmas nada contra Maria Rita So a minha opniao', 'Propaganda massa, bom ver a Elis, mas a música... nada a ver com o contexto😂', 'Fiquei ARREPIADA literalmente, que coisa mais linda, nunca vi propaganda mais bonita e mais bem produzida. Um estouro!! PARABÉNS, eterna Elis Regina.  🙌❤️', 'Muito bom. Consigo ver Elis com a filha adulta. Comerciais da Volkswagen sempre foram bons, mas esse, superou todos. 👏', 'a Elis Regina era contra a ditadura que vocês apoiaram ❤', 'A Elis do PS5 tá surreal. Kkkk', 'Parabéns por esse comercial emocionante! Voltei no tempo pra dentro do fusquinha do meu pai escutando Elis Regina nas fitas cassete dele que eu sempre desenrolava quando pegava hehehe. Me deu uma vontade imensa de ter uma "kombi" des